In [716]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain_core.documents.base import Document
from langchain.memory import ConversationBufferMemory

from dotenv import load_dotenv
from vertexai.generative_models import (
    SafetySetting,
    HarmCategory,
    HarmBlockThreshold,
)

load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [717]:
messages = []

def convert_to_str(messages):
    ans = ""
    for i in messages:
        ans += i[0] + " : " + i[1] + "\n"

    return ans

In [718]:
safety = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_UNSPECIFIED,
        threshold=HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_NONE
    )
]

In [719]:
model = ChatGoogleGenerativeAI(
    model="gemini-pro",
    client=genai,
    temperature=0.5,
    safety_settings=None
)

embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001")

new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True) 

user_data = """
    USER DATA
    ```
        phone_no 932489237
        name "Raj Patel"
        email "raja23@gmail.com"
        subscriptionStatus false

        previousOrders ->
        order_id "ord123"
        status: "Delivered"
        product: 
        name "HRX Oversized T-Shirt"
        description "Cotton-Comfy fit Oversized T-Shirt"
        category "Clothing-Men-TShirt"
        average_rating 3
        price 399
    ```
"""

first_context = """
    Your name is Radhika from Amazon Customer Support Agent Team and you will be helping a customer today. 

    You will be speaking, so output text accordingly. 

    Do not verify anything about user because he is already verified.
    
    DO NOT SHARE USER DATA TO THE USER.

    You should never ask them to contact Amazon Customer service, instead if you are unable to find the answer, forward the call to agent and terminate by [AGENT]
"""

initial_prompt_template = """
    {first_context}

    {context}

    User Question : Only try to answer this and add nothing else.
    {user_input}

    {user_data}

    Previous Chat History:
    This data below only shows the history of the conversation. Do not replicate it. Always answer according to User Input.
    {chat_history}

    WRITE [TERMINATE] IF THE USER IS SATISFIED
    IF UNABLE TO FIND THE USER ANSWER IN CONTEXT, TRANSFER THE CALL TO AGENT BY [AGENT]

    {ending_lines}
"""

prompt = PromptTemplate(
    input_variables=["context", "user_input", "first_context", "user_data", "ending_lines", "chat_history"],
    template=initial_prompt_template
)

chain = load_qa_chain(llm=model, chain_type="stuff", prompt = prompt)

In [720]:
def filter_contexts(term):
    contexts = new_db.similarity_search_with_score(term, k=3)

    ans_contexts = []

    for i in contexts:
        print(i[1])
        if(i[1] > 0.5):
            ans_contexts.append(i[0])

    return ans_contexts

In [721]:
#First Input
user_question = "Hi, I placed an order last week and I haven't received it yet. Can you help me find out the status of my order?"
context_user = filter_contexts(user_question)

first_message = initial_prompt_template.format(first_context = first_context, input_documents = context_user, user_input = user_question, user_data = user_data, ending_lines = "Your call is now live with the agent! Do not include anything of previous chat in replies. Use them to only generate future answers.", context = '', chat_history = '')

res = chain({"first_context": first_context, 
"input_documents": context_user, 
"user_input":  user_question, 
"user_data": "User Data \n" + user_data, 
"ending_lines": "Begin! Call is live with customer", 
"chat_history": convert_to_str(messages)}, return_only_outputs=True)

messages.append(("system", first_message))
messages.append(("agent", res['output_text']))
res

0.4268624
0.5001756
0.50114465


{'output_text': "I'm sorry to hear that you haven't received your order yet. I can help you find out the status of your order. Could you please provide me with your order number?"}

In [715]:
user_question = """It is ORD-123"""
context_user = filter_contexts(user_question)

response = chain({
    "first_context": "", 
    "input_documents": context_user, 
    "user_input": user_question, 
    "user_data": user_data, 
    "ending_lines": "", 
    "chat_history": convert_to_str(messages)
}, return_only_outputs = True)

messages.append(("human", user_question))
messages.append(("agent", response['output_text']))
response

0.64912856
0.69469887
0.71312106


{'output_text': 'It is ORD-123'}

In [651]:
user_question = "Okay, can you tell me something about me!"
context_user = new_db.similarity_search(user_question)

response = chain({
    "first_context": "",
    "input_documents": "",
    "user_input":  user_question,
    "user_data": "", "ending_lines": "",
    "chat_history": convert_to_str(messages)
}, return_only_outputs = True)

messages.append(("human", user_question))
messages.append(("agent", response['output_text']))

In [652]:
response

{'output_text': '[TERMINATE]\nThank you for calling Amazon!'}

In [653]:
user_question = "Thank you for the answer!"
context_user = filter_contexts(user_question)

response = chain({
    "first_context": "",
    "input_documents": context_user,
    "user_input":  user_question,
    "user_data": "", "ending_lines": "",
    "chat_history": convert_to_str(messages)
}, return_only_outputs = True)

messages.append(("human", user_question))
messages.append(("agent", response['output_text']))

0.5054102
0.57055926
0.6090071


In [654]:
response

{'output_text': 'Are you satisfied with my service?\n[TERMINATE]\nThank you for calling Amazon!'}